In [42]:
#import necessary libraries for data extraction, cleaning, visualization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#import library for api calls
import requests

#import library for using our csv files
import os

In [43]:
#get league ids

league1_id = "732757906084204544"
league2_id = "784371753404203008"
league3_id = "918364211628253184"
league4_id = "1050224316085932032"

leagues = [league1_id, league2_id, league3_id, league4_id]

In [44]:
#create function for extracting sleeper data base on url
def extract(url):
    
    r = requests.get(url)

    r.raise_for_status()
        
    json = r.json()
        
    df = pd.DataFrame(json)
    
    return df

In [45]:
#get users in my league
url = f"https://api.sleeper.app/v1/league/{league1_id}/users"

user_df = extract(url)

#we only care about id, team name, and username
user_df = user_df[["user_id", "display_name", "metadata"]]

user_df["metadata"] = user_df["metadata"].apply(lambda x: x.get("team_name"))

user_df["metadata"] = user_df.apply(lambda row: f"Team {row['display_name']}" if row["metadata"] is None else row["metadata"],
    axis=1)

user_df = user_df.rename(columns={"display_name": "username", "metadata":"team_name"})

In [46]:
#change team names and usernames for privacy
nums = range(1,11)
default_users = []
default_teams = []

for num in nums:
    default_users.append(f"User {num}")
    default_teams.append(f"Team {num}")
    
user_df["username"] = default_users
user_df["team_name"] = default_teams

user_df

,user_id,username,team_name
0,80863790908194816,User 1,Team 1
1,603675480201166848,User 2,Team 2
2,732792654391291904,User 3,Team 3
3,732793182814871552,User 4,Team 4
4,732822827383771136,User 5,Team 5
5,732987287658409984,User 6,Team 6
6,733710981288591360,User 7,Team 7
7,733748080079368192,User 8,Team 8
8,733774337605251072,User 9,Team 9
9,733798916314746880,User 10,Team 10


In [47]:
#get drafts in my league
drafts_df = pd.DataFrame()

for league_id in leagues: # type: ignore

    url = f"https://api.sleeper.app/v1/league/{league_id}/drafts"

    draft_df = extract(url)

    drafts_df = pd.concat([drafts_df,draft_df], ignore_index=True)
    
#only need draft id, keep year as well
drafts_df = drafts_df[["draft_id", "season"]]

drafts_df

,draft_id,season
0,732757906650386432,2021
1,784371753404203009,2022
2,918364211628253185,2023
3,1050224316085932033,2024


In [48]:
#get draft picks for each draft
picks_df = pd.DataFrame()

for draft in drafts_df["draft_id"]:
    
    url = f"https://api.sleeper.app/v1/draft/{draft}/picks"
    
    picks = extract(url)
    
    picks_df = pd.concat([picks_df,picks], ignore_index=True)

#get player name from metadata
picks_df["metadata"] = picks_df["metadata"].apply(lambda x: x["first_name"] + " " + x["last_name"])
    
#keep relevant data
picks_df = picks_df[["player_id","metadata","picked_by","round","pick_no","draft_id"]]

#rename picked_by to user_id and metadata to Player so we can join on them later
picks_df = picks_df.rename(columns={"picked_by" : "user_id", "metadata":"Player"})

picks_df


,player_id,Player,user_id,round,pick_no,draft_id
0,4046,Patrick Mahomes,603675480201166848,1,1,732757906650386432
1,4984,Josh Allen,733748080079368192,1,2,732757906650386432
2,4029,Dalvin Cook,732822827383771136,1,3,732757906650386432
3,4034,Christian McCaffrey,732987287658409984,1,4,732757906650386432
4,4881,Lamar Jackson,733798916314746880,1,5,732757906650386432
...,...,...,...,...,...,...
355,11596,Ben Sinnott,603675480201166848,3,26,1050224316085932033
356,11617,Malachi Corley,732792654391291904,3,27,1050224316085932033
357,11600,Ja'Tavion Sanders,733710981288591360,3,28,1050224316085932033
358,11645,Javon Baker,732793182814871552,3,29,1050224316085932033


In [49]:
#scrape pro football reference
fantasy_stats = pd.DataFrame()

seasons = ["2021", "2022", "2023"]

#scrape only once
if os.path.exists("fantasy_stats.csv"):
    fantasy_stats = pd.read_csv("fantasy_stats.csv", index_col=0)
else:
    for season in seasons:
    
        url = f"https://www.pro-football-reference.com/years/{season}/fantasy.htm"
        
        fantasy_season = pd.read_html(url, header=1, attrs={'id':'fantasy'})[0]
        
        fantasy_stats = pd.concat([fantasy_stats, fantasy_season], ignore_index=True)
    
    fantasy_stats.to_csv("fantasy_stats.csv")
    
fantasy_stats

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,TD.3,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
0,1,Jonathan Taylor*+,IND,RB,22,17,17,0,0,0,...,20,NaN,NaN,333,373.1,381.1,353.1,176,1,1
1,2,Cooper Kupp*+,LAR,WR,28,17,17,0,1,0,...,16,1,NaN,295,439.5,442.5,367.0,163,1,2
2,3,Deebo Samuel*+,SFO,WR,25,16,15,1,2,24,...,14,NaN,NaN,262,339.0,347.0,300.5,132,2,3
3,4,Josh Allen,BUF,QB,25,17,17,409,646,4407,...,6,2,1,403,402.6,426.6,417.6,126,1,4
4,5,Austin Ekeler,LAC,RB,26,16,16,0,0,0,...,20,2,NaN,274,343.8,352.8,308.8,120,2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2035,629,Deon Jackson,3TM,RB,24,4,1,0,0,0,...,0,NaN,NaN,-1,4.0,6.0,1.5,NaN,166,NaN
2036,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,TD,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank
2037,630,David Wells,TAM,TE,28,5,0,0,0,0,...,0,NaN,NaN,-1,1.0,1.0,NaN,NaN,140,NaN
2038,631,James Proche,CLE,WR,27,10,1,0,0,0,...,0,NaN,NaN,-2,-2.0,-1.0,-2.0,NaN,242,NaN


In [50]:
#lets just look at name, team, position, age, games played, and fantasy stats for now
selected_columns = fantasy_stats.loc[:, "Player":"G"].columns.to_list() + fantasy_stats.loc[:, "FantPt":].columns.to_list()

fantasy = fantasy_stats.loc[:, selected_columns].copy()

#and we have to join on player name so we have to get rid of * and + indicating pro bowl or something like that
fantasy["Player"] = fantasy["Player"].apply(lambda x: x[:-2] if "*+" in x else x)
fantasy["Player"] = fantasy["Player"].apply(lambda x: x[:-1] if "*" in x else x)

#drop rows that aren't stats (you can see in the table in pro football reference)
fantasy = fantasy[fantasy["Player"] != "Player"]

#finally, we use half-ppr in our league so we can add that and get rid of the other point systems
#standard is rounded and ppr is not so it wont be perfect

fantasy["Half_PPR"] = round(fantasy[['FantPt', 'PPR']].astype(float).mean(axis=1), 1)

fantasy = fantasy.drop(columns=["FantPt","PPR","DKPt","FDPt"])

fantasy

,Player,Tm,FantPos,Age,G,VBD,PosRank,OvRank,Half_PPR
0,Jonathan Taylor,IND,RB,22,17,176,1,1,353.0
1,Cooper Kupp,LAR,WR,28,17,163,1,2,367.2
2,Deebo Samuel,SFO,WR,25,16,132,2,3,300.5
3,Josh Allen,BUF,QB,25,17,126,1,4,402.8
4,Austin Ekeler,LAC,RB,26,16,120,2,5,308.9
...,...,...,...,...,...,...,...,...,...
2034,Kyle Allen,BUF,QB,27,7,NaN,83,NaN,-1.2
2035,Deon Jackson,3TM,RB,24,4,NaN,166,NaN,1.5
2037,David Wells,TAM,TE,28,5,NaN,140,NaN,0.0
2038,James Proche,CLE,WR,27,10,NaN,242,NaN,-2.0


In [51]:
#draft picks left join with fantasy stats on "Player"
df = picks_df.merge(fantasy, on="Player", how="left")

In [52]:
#we want the season too so join with drafts on draft id
df = df.merge(drafts_df, on="draft_id")

In [53]:
#finally, want username and team name so merge with users on user id
full_df = df.merge(user_df, on="user_id")

In [54]:
full_df.head()

,player_id,Player,user_id,round,pick_no,draft_id,Tm,FantPos,Age,G,VBD,PosRank,OvRank,Half_PPR,season,username,team_name
0,4046,Patrick Mahomes,603675480201166848,1,1,732757906650386432,KAN,QB,26,17,88,4,13,361.8,2021,User 2,Team 2
1,4046,Patrick Mahomes,603675480201166848,1,1,732757906650386432,KAN,QB,27,17,136,1,1,416.7,2021,User 2,Team 2
2,4046,Patrick Mahomes,603675480201166848,1,1,732757906650386432,KAN,QB,28,16,17,8,50,280.1,2021,User 2,Team 2
3,4984,Josh Allen,733748080079368192,1,2,732757906650386432,BUF,QB,25,17,126,1,4,402.8,2021,User 8,Team 8
4,4984,Josh Allen,733748080079368192,1,2,732757906650386432,BUF,QB,26,16,117,2,7,395.8,2021,User 8,Team 8


# Analysis

### We have our Dataframe so lets check some stats

1. Total Points Drafted by a Team
2. Points Per Draft Pick for a Team
3. AVG PPG Per Draft Pick for a Team
4. Above Stats for a specific seasons Ex: Who drafted best in 2021?
5. How many top ten overall / positional players has each user drafted?
6. Total VBD Drafted by a team (fantasy points - fantasy points for average player at their position)
7. Find the best player drafted by each team (by total points / ppg and overall / per season)



In [55]:
total_points = full_df.groupby(['username','team_name'])["Half_PPR"].sum().sort_values(ascending=False)
total_points

username  team_name
User 1    Team 1       11261.0
User 7    Team 7        9400.3
User 2    Team 2        9138.0
User 4    Team 4        9025.6
User 6    Team 6        8797.1
User 9    Team 9        7915.9
User 3    Team 3        7873.9
User 8    Team 8        7816.7
User 5    Team 5        7599.0
User 10   Team 10       6992.4
Name: Half_PPR, dtype: float64

In [56]:
total_picks = full_df[full_df["season"] != "2024"].groupby(['username','team_name'])["Player"].nunique().sort_values(ascending=False) #only for 2021-2023
total_picks

username  team_name
User 1    Team 1       43
User 3    Team 3       34
User 2    Team 2       33
User 7    Team 7       33
User 10   Team 10      32
User 5    Team 5       32
User 6    Team 6       32
User 9    Team 9       32
User 4    Team 4       31
User 8    Team 8       28
Name: Player, dtype: int64

In [57]:
points_per_pick = total_points / total_picks
points_per_pick.sort_values(ascending=False)

username  team_name
User 4    Team 4       291.148387
User 7    Team 7       284.857576
User 8    Team 8       279.167857
User 2    Team 2       276.909091
User 6    Team 6       274.909375
User 1    Team 1       261.883721
User 9    Team 9       247.371875
User 5    Team 5       237.468750
User 3    Team 3       231.585294
User 10   Team 10      218.512500
dtype: float64

In [58]:
#forgot to add ppg to df so lets do that
full_df["PPG"] = round(full_df["Half_PPR"] / full_df["G"].astype(float), 1)

In [59]:
ppg_per_pick = full_df.groupby(['username','team_name'])["PPG"].sum() / total_picks
ppg_per_pick.sort_values(ascending=False)

username  team_name
User 7    Team 7       20.312121
User 4    Team 4       20.238710
User 8    Team 8       20.221429
User 6    Team 6       19.662500
User 2    Team 2       19.484848
User 1    Team 1       19.248837
User 9    Team 9       18.490625
User 5    Team 5       17.434375
User 3    Team 3       17.079412
User 10   Team 10      15.381250
dtype: float64

In [60]:
points_per_draft = pd.DataFrame()

for season in ["2021", "2022", "2023"]:

    points_draft = full_df[full_df["season"] == season].groupby(['season','username','team_name'])["Half_PPR"].sum().sort_values(ascending=False)
    
    points_per_draft = pd.concat([points_per_draft,points_draft])

points_per_draft.sort_values(ascending=False, by="Half_PPR")

,Half_PPR
"(2021, User 1, Team 1)",9393.0
"(2021, User 2, Team 2)",8502.9
"(2021, User 7, Team 7)",8343.8
"(2021, User 4, Team 4)",8304.2
"(2021, User 6, Team 6)",8197.7
"(2021, User 9, Team 9)",7404.5
"(2021, User 8, Team 8)",7337.3
"(2021, User 5, Team 5)",7095.9
"(2021, User 3, Team 3)",6689.3
"(2021, User 10, Team 10)",6306.3


In [61]:
#count number draft picks that had top ten overall seasons
full_df['OvRank'] = pd.to_numeric(full_df['OvRank'], errors='coerce')
full_df[full_df["OvRank"] < 11].groupby(["username","team_name"])["Player"].count().sort_values(ascending=False)

username  team_name
User 6    Team 6       5
User 2    Team 2       4
User 4    Team 4       4
User 7    Team 7       4
User 8    Team 8       4
User 10   Team 10      3
User 1    Team 1       2
User 5    Team 5       2
User 9    Team 9       2
Name: Player, dtype: int64

In [62]:
#count number draft picks that had top ten positional seasons
full_df['PosRank'] = pd.to_numeric(full_df['PosRank'], errors='coerce')
full_df[full_df["PosRank"] < 11].groupby(["username","team_name"])["Player"].count().sort_values(ascending=False)

username  team_name
User 2    Team 2       16
User 6    Team 6       15
User 7    Team 7       15
User 10   Team 10      12
User 1    Team 1       11
User 4    Team 4       11
User 5    Team 5        9
User 9    Team 9        9
User 8    Team 8        8
User 3    Team 3        6
Name: Player, dtype: int64

In [63]:
full_df['VBD'] = pd.to_numeric(full_df['VBD'], errors='coerce')
full_df.groupby(["username","team_name"])["VBD"].sum().sort_values(ascending=False)

username  team_name
User 7    Team 7       1368.0
User 2    Team 2       1164.0
User 6    Team 6       1050.0
User 1    Team 1       1026.0
User 4    Team 4        960.0
User 8    Team 8        815.0
User 5    Team 5        779.0
User 10   Team 10       756.0
User 9    Team 9        553.0
User 3    Team 3        375.0
Name: VBD, dtype: float64

In [64]:
unique_players = full_df.groupby("Player")["Half_PPR"].sum().to_frame().merge(full_df, on="Player", how="left")

unique_players.groupby(["username","team_name"]).apply(lambda x: x.loc[x["Half_PPR_x"].idxmax()])[["Player","Half_PPR_x"]].sort_values(by="Half_PPR_x",ascending=False)


/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_45825/2858628358.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  unique_players.groupby(["username","team_name"]).apply(lambda x: x.loc[x["Half_PPR_x"].idxmax()])[["Player","Half_PPR_x"]].sort_values(by="Half_PPR_x",ascending=False)


,,Player,Half_PPR_x
username,team_name,,
User 8,Team 8,Josh Allen,1191.4
User 2,Team 2,Patrick Mahomes,1058.6
User 4,Team 4,Jalen Hurts,1047.0
User 9,Team 9,Dak Prescott,862.5
User 7,Team 7,Tyreek Hill,845.5
User 5,Team 5,Joe Burrow,812.0
User 10,Team 10,Lamar Jackson,807.1
User 6,Team 6,Christian McCaffrey,780.5
User 1,Team 1,CeeDee Lamb,777.3


In [65]:
full_df.groupby(["username","team_name"]).apply(lambda x: x.loc[x["PPG"].idxmax()])[["Player","PPG"]].sort_values(by="PPG",ascending=False)

/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_45825/3585566620.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  full_df.groupby(["username","team_name"]).apply(lambda x: x.loc[x["PPG"].idxmax()])[["Player","PPG"]].sort_values(by="PPG",ascending=False)


,,Player,PPG
username,team_name,,
User 4,Team 4,Jalen Hurts,25.2
User 8,Team 8,Josh Allen,24.7
User 2,Team 2,Patrick Mahomes,24.5
User 7,Team 7,Derrick Henry,23.0
User 6,Team 6,Christian McCaffrey,22.4
User 10,Team 10,Tom Brady,22.0
User 5,Team 5,Joe Burrow,21.9
User 9,Team 9,Dak Prescott,20.2
User 1,Team 1,CeeDee Lamb,19.7


In [66]:
best_players_per_draft = pd.DataFrame()

for season in ["2021", "2022", "2023"]:

    best_player_in_draft = unique_players[unique_players["season"] == season].groupby(["username","team_name"]).apply(lambda x: x.loc[x["Half_PPR_x"].idxmax()])[["season","Player","Half_PPR_x"]].sort_values(by="Half_PPR_x",ascending=False)
    
    best_players_per_draft = pd.concat([best_players_per_draft,best_player_in_draft])
    
best_players_per_draft.sort_values(by="Half_PPR_x",ascending=False)

/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_45825/1682431741.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_player_in_draft = unique_players[unique_players["season"] == season].groupby(["username","team_name"]).apply(lambda x: x.loc[x["Half_PPR_x"].idxmax()])[["season","Player","Half_PPR_x"]].sort_values(by="Half_PPR_x",ascending=False)
/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_45825/1682431741.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings o

,,season,Player,Half_PPR_x
username,team_name,,,
User 8,Team 8,2021,Josh Allen,1191.4
User 2,Team 2,2021,Patrick Mahomes,1058.6
User 4,Team 4,2021,Jalen Hurts,1047.0
User 9,Team 9,2021,Dak Prescott,862.5
User 7,Team 7,2021,Tyreek Hill,845.5
User 5,Team 5,2021,Joe Burrow,812.0
User 10,Team 10,2021,Lamar Jackson,807.1
User 6,Team 6,2021,Christian McCaffrey,780.5
User 1,Team 1,2021,CeeDee Lamb,777.3


In [67]:
best_players_per_draft = pd.DataFrame()

for season in ["2021", "2022", "2023"]:

    best_player_in_draft = full_df[full_df["season"] == season].groupby(["username","team_name"]).apply(lambda x: x.loc[x["PPG"].idxmax()])[["season","Player","PPG"]].sort_values(by="PPG",ascending=False)
    
    best_players_per_draft = pd.concat([best_players_per_draft,best_player_in_draft])
    
best_players_per_draft.sort_values(by="PPG",ascending=False)

/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_45825/1671719336.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  best_player_in_draft = full_df[full_df["season"] == season].groupby(["username","team_name"]).apply(lambda x: x.loc[x["PPG"].idxmax()])[["season","Player","PPG"]].sort_values(by="PPG",ascending=False)
/var/folders/fd/wjhkh2294ws3fn6xqxwlk5qw0000gn/T/ipykernel_45825/1671719336.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping co

,,season,Player,PPG
username,team_name,,,
User 4,Team 4,2021,Jalen Hurts,25.2
User 8,Team 8,2021,Josh Allen,24.7
User 2,Team 2,2021,Patrick Mahomes,24.5
User 7,Team 7,2021,Derrick Henry,23.0
User 6,Team 6,2021,Christian McCaffrey,22.4
User 10,Team 10,2021,Tom Brady,22.0
User 5,Team 5,2021,Joe Burrow,21.9
User 9,Team 9,2021,Dak Prescott,20.2
User 1,Team 1,2021,CeeDee Lamb,19.7


# Future Plans

1. Add plots for the stats we have gathered so far
2. Add more analysis / Answer more questions
3. Add more markdown explaining the code
4. Save to CSV and create a Tableau Dashboard based on the df we created
5. Fix Deprecation warnings